<a href="https://colab.research.google.com/github/kgpark88/ems/blob/master/energy_usage_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전력 소비량 예측 딥러닝 모델 개발 및 검증

### 필요한 라이브러리 로드

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from google.colab import auth
from google.colab import drive

### 데이터셋을 입력 데이터와 타깃 데이터로 분리합니다.

In [0]:
def split_univariate_data(dataset, start_index, end_index, hist_data_size, target_size):
    data = []
    labels = []

    start_index = start_index + hist_data_size
    if end_index is None:
        # end_index = len(dataset) - target_size
        end_index = int(start_index + start_index/2)

    for i in range(start_index, end_index):
        indices = range(i-hist_data_size, i)
        # Reshape data from (hist_data_size,) to (hist_data_size, 1)
        data.append(np.reshape(dataset[indices], (hist_data_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

### 전력사용량 예측을 위한 딥러닝모델의 입력 데이터는 파란색점으로 표시하고, 타깃값은 빨간색 십자로 표시합니다.

In [0]:
def show_plot(data, delta, title):
    labels = ['Previous Power Usage', 'Future Power Usage', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    data_count = data[0].shape[0]
    time_steps = [i for i in range(-data_count, 0, 1)]
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)

    for i, v in enumerate(data):
        if i:
            plt.plot(future, data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')     
    return plt

In [0]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, c='royalblue', label='Training loss',
                 lw=2,  ls="-", marker="o", ms=10, mew=3)
    plt.plot(epochs, val_loss, c='orangered', label='Validation loss',
                 lw=2, ls="--", marker="x", ms=10, mew=3)
    plt.title(title)
    plt.legend()
    plt.show()

### 데이터 로드를 위해 구글 드라이브를 연동합니다.

In [0]:
drive.mount("/content/gdrive")

### 데이터를 로딩합니다.

In [0]:
colab_dir ='/content/gdrive/My Drive/Colab Notebooks'
file_name = 'power_usage_201.csv'
data_file = os.path.join(colab_dir, "data", file_name)

df = pd.read_csv(data_file, header = 0, delimiter = ',', quoting = 3)
print('전체 데이터의 개수: {}'.format(len(df)))
df.index = df['daq_time']

In [0]:
df.head()

In [0]:
df.tail()

In [0]:
df_week = df.iloc[0:96*7]
df_week["power_usage"].plot()

* training datasetr과 validation dataset으로 분리 합니다. 
* training 데이터 70,080개는 2년치 데이터에 해당합니다. 
* 4개x24시x365일x3년 = 70,080개

### 재현성을 보장하기 위해 시드 설정.

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

## 일변량 시계열 예측
먼저 단일 피쳐 (전력사용량) 만 사용하여 모델을 학습하고, 향후 해당 값을 예측하는 데 사용합니다.

먼저 데이터세트에서 전력사용량만 추출합니다.

In [0]:
data = df['power_usage']
data.index = df['daq_time']
data.head()

시간에 따른 전력사용량 패턴을 분석합니다.

In [0]:
data.plot(subplots=True)

In [0]:
data = data.values

* 신경망을 training하기 전에, 먼저 training dataset의 피처(feature)를 정규화합니다.
* 정규화 방법은 각 피처에서 평균을 빼고 각 피처의 표준편차로 나눕니다.
* 상이한 스케일을 가진 값을 신경망에 주입하면 학습을 어렵게 만듭니다. 
* 이런 데이터는 특성별로 정규화(표준화)가 필요합니다. 
* 각 특성을 특성의 평균을 빼고 표준 편차로 나눕니다. 
* 그렇다면 특성의 평균은 0, 표준 편차는 1인 데이터셋으로 만들어집니다. 
* 본 정규화는 훈련 데이터 값만 이용해야 함을 주의해야합니다.

In [0]:
train_data_mean = data[:TRAIN_SPLIT].mean()
train_data_std = data[:TRAIN_SPLIT].std()
data = (data - train_data_mean)/train_data_std
data = np.round(data, 2)

###  일변량 모델에 대한 데이터를 만듭니다. 
모델에 20 개의 전력사용량 측정치가 제공되며,
다음 단계에서 전력사용량을 예측하는 방법을 학습합니다.

In [0]:
TRAIN_SPLIT = 70080
TEST_SPLIT = 105120

HISTORY_DATA_SIZE = 20
FUTURE_TARGET = 0

x_train, y_train = split_univariate_data(data, 0, TRAIN_SPLIT, 
                                         HISTORY_DATA_SIZE, 
                                         FUTURE_TARGET)
x_validation, y_validation = split_univariate_data(data, TRAIN_SPLIT, TEST_SPLIT, 
                                                     HISTORY_DATA_SIZE, 
                                                     FUTURE_TARGET)
x_test, _ = split_univariate_data(data, TEST_SPLIT, TEST_SPLIT+100, 
                                                     HISTORY_DATA_SIZE, 
                                                     FUTURE_TARGET)

split_data 함수가 반환하는 내용입니다.

In [0]:
print ('모델 입력 데이터 : 이전 전력사용량')
print (x_train[0])
print ('타겟 데이터 :  예측할 전력사용량')
print (y_train[0])

In [0]:
show_plot([x_train[0], y_train[0]], 0, 'Future Power Usage')

### 순환신경망(RNN : Recurrent Neural Network)
* 순환신경망은 시계열 데이터에 적합한 신경망입니다.
* 본 연구에서는 장단기 기억이 가능한 LSTM(Long Short Term Memory)이라는 RNN Layer를 사용합니다.




In [0]:
x_train.shape[-2:]

In [0]:
BATCH_SIZE = 96
BUFFER_SIZE = 10000

train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

validation = tf.data.Dataset.from_tensor_slices((x_validation, y_validation))
validation = validation.batch(BATCH_SIZE).repeat()

### 모델을 구성합니다.
전력사용량 데이터는 일 단위의 주기적인 추세가 있으므로 BATCH_SIZE는 96(15분*24시간=96)으로 하는 것이 적절합니다. 

In [0]:
HIDDEN_SIZE = 20
EPOCHS = 10

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(HIDDEN_SIZE, 
                               input_shape=(HISTORY_DATA_SIZE, 1),
                               return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

### 모델을 구성한 후 compile 메서드를 호출하여 학습 과정을 설정합니다.

In [0]:
model.compile(loss="mean_squared_error", optimizer="adam",
              metrics=["mean_squared_error", "mean_absolute_error"])

### 모델은 fit 메서드를 통해서 훈련 데이터를 학습합니다.
* steps_per_epoch은 한 epoch에 사용한 스텝수
* steps_per_epoch = train_size / batch_size  = 69600/96 = 725
* validation_steps는 한 epoch 종료시마다 검증할 때 사용되는 검증 스텝수
* validation_steps = validation / batch size  = 34608/96 = 360


In [0]:
# STEPS_PER_EPOCH = 200
# VALIDATION_STEPS = 50
STEPS_PER_EPOCH = 725
VALIDATION_STEPS = 360

history = model.fit(train, epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          validation_data=validation, validation_steps=VALIDATION_STEPS)

In [0]:
loss_and_metrics = model.evaluate(x_validation, y_validation)
print(loss_and_metrics)

In [0]:
plot_train_history(history, 'Single Step Training and validation loss')

#### LSTM model로 전력사용량을 예측합니다.

In [0]:
for i in range(0,10):
    input_data = x_validation[i:i+1,:,:]
    prediction = model.predict(input_data,  batch_size=1)
    prediction_value = prediction[0][0]
    power_usage = np.array([y_validation[i]])
    print("Prediction Value : {:.2f}, Power Usage : {:.2f}".format(prediction_value*train_data_std+train_data_std, power_usage[0]*train_data_std+train_data_std))
    plot = show_plot([input_data[0], power_usage, prediction_value], 0, 'Simple LSTM model')
    plot.show()


In [0]:
x = model.predict(x_train[:,:,:])*train_data_std+train_data_mean
y = y_train*train_data_std+train_data_mean

plt.plot(x, 'r--', label="Prediction Value")
plt.plot(y, 'b-', label="Power Usage")
plt.xlim(1, 96*31)
plt.legend()
plt.title("15min Power Usage(Month)")
plt.show()

In [0]:
plt.plot(x, 'r--', label="Prediction Value")
plt.plot(y, 'b-', label="Power Usage")
plt.xlim(96*200, 96*201)
plt.legend()
plt.title("15min Power Usage(Day)")
plt.show()

In [0]:
x.flatten()

In [0]:
y

### 모델 저장하기

In [0]:
m_name = os.path.splitext(file_name)[0]
model_json = model.to_json()
with open(colab_dir + '/model/' + m_name + '.json', 'w') as f : 
    f.write(model_json)
model.save_weights(colab_dir + '/model/' + m_name + '.h5')
print("Saved model to disk")

저장한 모델 불러오기

In [0]:
json_file = open(colab_dir +'/model/' + m_name + '.json', 'r')
loaded_model_json = json_file.read() 
json_file.close()
simple_lstm_model = tf.keras.models.model_from_json(loaded_model_json)
simple_lstm_model.load_weights(colab_dir +'/model/' + m_name + '.h5'")
print("Loaded model from disk")

#### 저장한 model을 사용하여 전력사용량 예측

In [0]:
for i in range(0,10):
    input_data = x_test[i:i+1,:,:]
    prediction = simple_lstm_model.predict(input_data,  batch_size=1)
    prediction_value = prediction[0][0]
    power_usage = np.array([y_validation[i]])
    print("Prediction Value : {:.2f}".format(prediction_value*train_data_std+train_data_mean))